# Постановка задачи.

Решение задачи кластеризации с использованием трех методов:
  1. KMeans
  2. DBSCAN
  3. Agglomerative Clustering

Шаг 1. Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import adjusted_rand_score

data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


Шаг 2. Подготовка данных

Кодирование категориальных признаков и масштабирование числовых

In [2]:
data_encoded = data.copy()

label_encoders = {}
for col in data_encoded.select_dtypes(include='object').columns:
    if col != 'income':
        label_encoders[col] = LabelEncoder()
        data_encoded[col] = label_encoders[col].fit_transform(data_encoded[col])

scaler = StandardScaler()
numerical_cols = data_encoded.select_dtypes(include='int64').columns
data_encoded[numerical_cols] = scaler.fit_transform(data_encoded[numerical_cols])

Шаг 3. Реализация методов кластеризации

KMeans

In [8]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_labels = kmeans.fit_predict(data_encoded.drop(columns=['income']))

ari_kmeans = adjusted_rand_score(LabelEncoder().fit_transform(data['income']), kmeans_labels)
print(f'Adjusted Rand Index для KMeans: {ari_kmeans}')
print(f'Количество кластеров для KMeans: {len(set(kmeans_labels))}')

Adjusted Rand Index для KMeans: -0.006918496755986382
Количество кластеров для KMeans: 5


DBSCAN

In [18]:
dbscan = DBSCAN(eps=2.0, min_samples=50)
dbscan_labels = dbscan.fit_predict(data_encoded.drop(columns=['income']))

ari_dbscan = adjusted_rand_score(LabelEncoder().fit_transform(data['income']), dbscan_labels)
print(f'Adjusted Rand Index для DBSCAN: {ari_dbscan}')
print(f'Количество кластеров для DBSCAN: {len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)}') 


Adjusted Rand Index для DBSCAN: -0.0060328080207864105
Количество кластеров для DBSCAN: 6


Agglomerative Clustering

In [10]:
agglo = AgglomerativeClustering(n_clusters=5)
agglo_labels = agglo.fit_predict(data_encoded.drop(columns=['income']))

ari_agglo = adjusted_rand_score(LabelEncoder().fit_transform(data['income']), agglo_labels)
print(f'Adjusted Rand Index для Agglomerative Clustering: {ari_agglo}')
print(f'Количество кластеров для Agglomerative Clustering: {len(set(agglo_labels))}')

Adjusted Rand Index для Agglomerative Clustering: 0.11353725481301286
Количество кластеров для Agglomerative Clustering: 5


# Вывод

* KMeans: Метод кластеризации с 5 кластерами показал значение Adjusted Rand Index (-0.0069), что указывает на довольно слабую связь между результатами кластеризации и истинными метками. Это свидетельствует о том, что KMeans не смог выявить четкую структуру данных, хотя алгоритм все же попытался разделить их на несколько групп. Возможно, стоит пересмотреть выбор признаков или попробовать другие методы для улучшения кластеризации, так как результат оставляет пространство для улучшений.

* DBSCAN: В случае с DBSCAN, который обнаружил 6 кластеров, Adjusted Rand Index оказался равным -0.0060, что указывает на низкую согласованность кластеров с истинными метками. Этот результат также говорит о том, что DBSCAN не смог точно разделить данные на смысловые группы. Однако, учитывая, что DBSCAN не требует заранее заданного числа кластеров, результат может зависеть от выбранных параметров (например, значения eps и min_samples). Поэтому рекомендуется экспериментировать с этими параметрами, чтобы получить более адекватные результаты.

* Агломеративная кластеризация: С 5 кластерами агломеративная кластеризация показала наибольшее значение Adjusted Rand Index (0.1135) среди всех методов, что, хоть и является относительно низким значением, указывает на некоторую структурированность данных. Это может означать, что метод способен выделить несколько значимых групп в данных, хотя результат всё же далек от идеала. Для улучшения качества кластеризации можно попробовать пересмотреть параметры или применить другие методы предобработки данных.

Все три метода кластеризации продемонстрировали недостаточно высокие результаты в плане соответствия с истинными метками, что говорит о том, что данные либо плохо разделяются, либо выбранные алгоритмы не подходят для данной задачи. Стоит попробовать улучшить представление данных, изменить параметры алгоритмов или даже рассмотреть другие методы кластеризации. В частности, для DBSCAN может быть полезно попробовать подобрать параметры eps и min_samples с целью получения более точных кластеров, а для KMeans и агломеративной кластеризации — улучшить выбор признаков и применить другие способы нормализации данных.